In [ ]:
from ipyprogressivis.widgets.scatterplot import Scatterplot
sc = Scatterplot(enable_centroids=True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""
Clustering datasets may be found at
http://cs.joensuu.fi/sipu/datasets/
"""
from progressivis.cluster import MBKMeans
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
from progressivis.stats.api import RandomPTable
import numpy as np
from progressivis.datasets.random import generate_random_multivariate_normal_csv as gen_csv
from progressivis.stats.blobs_table import BlobsPTable, MVBlobsPTable

MULTIVARIATE_NORMAL = True

if not MULTIVARIATE_NORMAL: # simple isotropic blobs (using scikit-learn make_blobs)
    centers = [(0.1, 0.3), (0.7, 0.5), (-0.4, -0.3)]
    data = BlobsPTable(columns=['_0', '_1'],  centers=centers, cluster_std=0.2, rows=1000_000)
else: # i.e. control each cluster's covariance matrix (using numpy.random.multivariate_normal)
    means = [0.1, 0.3], [0.7, 0.5], [-0.4, -0.3]
    covs = [[0.01, 0], [0, 0.09]], [[0.04, 0], [0, 0.01]], [[0.09, 0.04], [0.04, 0.02]]
    data = MVBlobsPTable(columns=['_0', '_1'], means=means, covs=covs, rows=1000_000)
mbkmeans = MBKMeans(n_clusters=3, batch_size=100, tol=0.01, is_input=False)
sp = MCScatterPlot(classes=[('Scatterplot', '_0', '_1', mbkmeans)])
sp.create_dependent_modules(data,'result')
sp['Scatterplot'].min_value.result.update({'_0': -np.inf, '_1': -np.inf})
sp['Scatterplot'].max_value.result.update({'_0': np.inf, '_1': np.inf})
mbkmeans.create_dependent_modules(sp['Scatterplot'].range_query_2d)
sp.move_point = mbkmeans.dep.moved_center # for input management
# Create the dashboard object
show = sc.link_module(sp)
sp.scheduler.task_start();

In [ ]:
sc

In [ ]:
import asyncio as aio
await aio.sleep(3)
show()